In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from datasets import load_dataset

In [6]:
from huggingface_hub import hf_hub_download

In [10]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\researcher\.huggingface\token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
wandb login --relogin

# Dataset & Features

In [12]:
train_dataset = load_dataset("amazon_polarity", split="train[:40000]")
dev_dataset = load_dataset("amazon_polarity", split="train[40000:50000]")
# test_dataset=load_dataset("HFFErica/steamreviews","1000nms-english-test-amazon_polarity.csv")
data_files = { "sentiment":"1000nms-english-test-amazon_polarity.csv"}
raw_dataset=load_dataset("HFFErica/steamreviews", data_files=data_files)


print(train_dataset)
print(dev_dataset)
# print(test_dataset)
print(raw_dataset)

Found cached dataset amazon_polarity (C:/Users/researcher/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)
Found cached dataset amazon_polarity (C:/Users/researcher/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)
Using custom data configuration HFFErica--steamreviews-fddaecb42c4ef175


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--steamreviews-fddaecb42c4ef175/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


D:\Software\Anaconda3\envs\pytorch\lib\site-packages\datasets\download\streaming_download_manager.py:697: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 40000
})
Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 10000
})
DatasetDict({
    sentiment: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 999
    })
})


In [13]:
train_dataset.features

{'label': ClassLabel(num_classes=2, names=['negative', 'positive'], id=None),
 'title': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None)}

In [14]:
raw_dataset["sentiment"].features

{'label': Value(dtype='int64', id=None),
 'title': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None)}

In [15]:
from datasets import Dataset, Value, ClassLabel, Features

features = raw_dataset["sentiment"].features.copy()
features["label"] = ClassLabel(names=["negative", "positive"])
def adjust_labels(batch):
    batch["label"] = [sentiment for sentiment in batch["label"]]
    return batch
label_dataset = raw_dataset.map(adjust_labels, batched=True, features=features)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
label_dataset["sentiment"].features

{'label': ClassLabel(num_classes=2, names=['negative', 'positive'], id=None),
 'title': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None)}

In [17]:
test_dataset=label_dataset["sentiment"]

In [18]:
print(train_dataset)
print(dev_dataset)
print(test_dataset)

Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 40000
})
Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 10000
})
Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 999
})


In [19]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
train_dataset[0]

  0%|          | 0/40 [00:00<?, ?ba/s]

{'label': 1,
 'title': 'Stuning even for the non-gamer',
 'content': 'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^',
 'labels': 1}

In [20]:
dev_dataset = dev_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# bert-tiny + tokenizer

In [21]:
model_id = 'prajjwal1/bert-tiny'
model = AutoModelForSequenceClassification.from_pretrained(model_id, 
            num_labels=train_dataset.features["label"].num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\researcher\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [22]:
MAX_LENGTH = 256
train_dataset = train_dataset.map(lambda e: tokenizer(e['content'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
dev_dataset = dev_dataset.map(lambda e: tokenizer(e['content'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['content'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

  0%|          | 0/40 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dev_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [24]:
train_dataset.features

{'label': ClassLabel(num_classes=2, names=['negative', 'positive'], id=None),
 'title': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [25]:
train_dataset[0]

{'labels': tensor(1),
 'input_ids': tensor([  101,  2023,  2614,  2650,  2001,  3376,   999,  2009, 23262,  1996,
         12411,  7301,  1999,  2115,  2568,  2061,  2092,  1045,  2052, 28667,
          8462,  4859,  2009,  2130,  2000,  2111,  2040,  5223,  6819,  2094,
          1012,  2208,  2189,   999,  1045,  2031,  2209,  1996,  2208, 10381,
          4948,  2080,  2892,  2021,  2041,  1997,  2035,  1997,  1996,  2399,
          1045,  2031,  2412,  2209,  2009,  2038,  1996,  2190,  2189,   999,
          2009, 10457,  2185,  2013, 13587,  9019,  2075,  1998,  3138,  1037,
          4840,  2121,  3357,  2007, 24665,  3686,  7334,  1998,  3969,  3993,
         19505,  1012,  2009,  2052, 17894,  3087,  2040, 14977,  2000,  4952,
           999,  1034,  1035,  1034,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  

In [26]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [27]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    learning_rate=3e-4,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    # load_best_model_at_end=True,
    # # eval_steps=100,
    # evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=dev_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

train_out = trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: title, content. If title, content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1875
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: ericat. Use `wandb log

Step,Training Loss
100,0.561500
200,0.403400
300,0.370400
400,0.344700
500,0.345000
600,0.332700
700,0.278800
800,0.249700
900,0.243300
1000,0.252400


Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json
Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-1500
Configuration saved in ./results\checkpoint-1500\config.json
Model weights saved in ./results\checkpoint-1500\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




# pipeline

In [28]:
model = model.cpu()

In [29]:
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [30]:
test_examples = raw_dataset["sentiment"]
test_examples[2]

{'label': 1,
 'title': "No Man's Sky",
 'content': '\n DAY 0 ReviewNo Man\'s SkyIt\'s a great game that i\'ve been waiting for years to come out .this game gives you the freedom to explore space how you want and even evere plant is not the same .the size of the game is to great and simulations in some soft the universe. If you want to see what is the game watch some Spoiler free videos or trailers ----------------------------Now to the bad part The day 0 Most People had problems with the game right at lanch but adding to that most of the people can\'t go and google "No Man\'s Sky Crash Fix" , and that they have to spam steam ,Any way don\'t belive what most people say steam reviews is Khos right now Yes the game might crash if you don\'t install everything correctly.Yes the game has some lag spikes but that can be fixed with few settings editing .No , this game has a multiplayer just it\'s rare to find players (and it\'ll work if you connect to the servers)No , Sean Didn\'t lie - most 

In [31]:
result = classifier(test_examples[2]['content'])
result

[{'label': 'LABEL_1', 'score': 0.6125274300575256}]

## Test on raw data

In [32]:
data_files = { "raw":"nms-english-105185-time.csv"}
df_dataset=load_dataset("HFFErica/steamreviews", data_files=data_files)
df_dataset

Using custom data configuration HFFErica--steamreviews-623717dbbfe38bf0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\datasets\download\streaming_download_manager.py:697: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--steamreviews-623717dbbfe38bf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    raw: Dataset({
        features: ['_id', 'time', 'content'],
        num_rows: 105184
    })
})

In [36]:
df_dataset["raw"][500]

{'_id': '5f6f537bd393fd0aa05959f3',
 'time': '12-Aug-16',
 'content': "\r\n\t\t\t\t\t\t\t\tA new update drastically improved the game and has made it more akin to the way it was presented in the original trailers. That being said, the core gameplay loop remains the same and if you're not a fan of it then chances are you still won't find any value in this.\t\t\t\t\t\t\t"}

In [37]:
results = classifier(df_dataset["raw"][500]['content'])
results

[{'label': 'LABEL_0', 'score': 0.8357300758361816}]